In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.gridspec as gridspec


In [2]:
input_dir = 'inputs'
output_dir = 'outputs'

In [3]:
def get_start(current_width, previous_width, previous_step):
    """
    Returns the appropriate start offset based on 
    """
    circle = np.pi*2 # 1 radian = 6.28
    if previous_width is None:
        previous_width = 0
    stepper = circle * 0.05  # okay honestly don't really know why this needs to be 0.05
    return previous_step + stepper * (current_width*10 + previous_width*10) 

def sunburst(widths, heights, labels, colors, output_file):
    """
    Make a sunburst pie chart (pie chart with different heights for each pie)
    """
    try:
        assert float(sum(widths)) == 1.0  # make sure all slices equal 1
    except AssertionError:
        print("Pie slices do not equal 1! Pie chart may be incomplete! {}, {}".format(output_file, sum(widths)))

    previous_step = 0
    previous_width = None

    starts = []
    for w in range(len(widths)):
        starts.append(get_start(widths[w], previous_width, previous_step))
        previous_step = get_start(widths[w], previous_width, previous_step)
        previous_width = widths[w]
    
    circle = np.pi*2 # 1 radian = 6.28
    widths = [circle*x for x in widths]
    
    fig = plt.figure(figsize=(10, 5))
    gs = gridspec.GridSpec(1, 2)
    ax = plt.subplot(gs[:,:1], projection='polar')
    legend_ax = plt.subplot(gs[:,1:])
    bars = ax.bar(starts, heights, widths, bottom=0.0)  # change bottom to 1 for 'donut' look

    # Use custom colors and opacity
    i = 0
    for r, bar in zip(heights, bars):
        bar.set_alpha(0.5)
        bar.set_color(colors[labels[i]])
        bar.set_linewidth(0)
        bar.set_label("{} w={} h={}".format(labels[i], widths[i], heights[i]))
        i += 1
    plt.setp(ax.get_xticklabels(), visible=False)
    # plt.setp(ax.get_yticklabels(), visible=False)
    # plt.show()
    set_legend(ax, legend_ax)
    fig.savefig(output_file)
    plt.cla()
    plt.clf()
    
def set_legend(ax, legend_ax):
    sns.despine(
        ax=legend_ax, 
        bottom=True, 
        top=True, 
        left=True,
        right=True
    )
    legend_ax.yaxis.set_visible(False)
    legend_ax.xaxis.set_visible(False)

    leg_handles, leg_labels = ax.get_legend_handles_labels()

    leg = legend_ax.legend(
        leg_handles, leg_labels, loc=10,
        mode="expand", ncol=1)

    # for legobj in leg.legendHandles:
    #     legobj.set_linewidth(4.0)

In [4]:
df = pd.read_table(
    os.path.join(input_dir, '20170505.ALLENCODEinclnotsubmitted.txt.nopipes.txt.submitted_only'),
    names=[
        'RBP','cell','element','uID','uID2',
        'ip_reads','ip_rpr',
        'input_','input_rpr','fold_change',
        'information_content'
    ],
    index_col=0
)
print(df.shape[0])
df.head()

65884


,RBP,cell,element,uID,uID2,ip_reads,ip_rpr,input_,input_rpr,fold_change,information_content
206_01_HNRNPK,HNRNPK,HepG2,Alu,206_01_HNRNPK,HNRNPK-HepG2_01,70957,0.014171,116845,0.016044,0.883225,-0.002539
206_02_HNRNPK,HNRNPK,HepG2,Alu,206_02_HNRNPK,HNRNPK-HepG2_02,73372,0.014843,116845,0.016044,0.925101,-0.001667
358_01_GNL3,GNL3,K562,Alu,358_01_GNL3,GNL3-K562_01,13374,0.003358,254075,0.012172,0.275841,-0.006239
358_02_GNL3,GNL3,K562,Alu,358_02_GNL3,GNL3-K562_02,29507,0.005196,254075,0.012172,0.426899,-0.006381
331_01_RBM22,RBM22,K562,Alu,331_01_RBM22,RBM22-K562_01,24679,0.009018,52744,0.006284,1.435128,0.004700


In [5]:
# 5% without fold chnge cutoff
# 1% with fold change cutoff of 4 or something
# fold change cutoff isn't log

In [6]:
def split_others(df, fold_change_cutoff, soft_cutoff, hard_cutoff):
    
    pass1 = df[
        (df['ip_rpr'] >= soft_cutoff) & \
        (df['fold_change'] >= 1)
    ]
    pass2 = df[
        (df['ip_rpr'] >= hard_cutoff) & \
        (df['fold_change'] >= fold_change_cutoff)
    ]
    passed = pd.Index(set(pass1.index).union(set(pass2.index)))
    failed = df.index.difference(passed)
    return df.loc[passed], df.loc[failed]
    
def get_pie_values(df, rbp, fold_change_cutoff=4, soft_cutoff=0.05, hard_cutoff=0.01):
    dx = df.loc[rbp][['element','ip_rpr','fold_change']]
    dx.set_index('element', inplace=True)
    dpass, dfail = split_others(dx, fold_change_cutoff, soft_cutoff, hard_cutoff)
    others = pd.DataFrame(columns=['ip_rpr','fold_change'],index=['others'])
    others.loc['others']['ip_rpr'] = dfail['ip_rpr'].sum()
    others.loc['others']['fold_change'] = dfail['fold_change'].mean() if dfail['fold_change'].mean() >=0 else 0
    merged = pd.concat([dpass, others])
    # because the ip_circular_... doesn't always add up to 1
    if merged['ip_rpr'].sum(axis=0) < 1:
        merged.loc['others', 'ip_rpr'] += (
            1 - merged['ip_rpr'].sum(axis=0)
        )
    return merged



In [7]:
# vals = get_pie_values(df, '204_01_RBFOX2')
# colors = sns.color_palette("hls", vals.shape[0])
# sunburst(vals['ip_rpr'], vals['fold_change'], vals.index, colors)

In [8]:
# vals = get_pie_values(df, '203_01_HNRNPC')
# colors = sns.color_palette("hls", vals.shape[0])
# sunburst(vals['ip_rpr'], vals['fold_change'], vals.index, colors)

In [9]:
from tqdm import trange, tqdm_notebook

def make_colordict(elements, color_fn):
    colors = {}
    with open(color_fn, 'r') as f:
        colors['others'] = f.readline().rstrip() # don't like black, it's the first color so... skip
        for element in elements:
            colors[element] = f.readline().rstrip()
    
    return colors

color_fn = os.path.join(input_dir, 'color_list_269.lines')
colors = make_colordict(set(df['element']), color_fn)
progress = trange(len(set(df.index)))

for rbp in set(df.index):
    vals = get_pie_values(df, rbp)

    output_file = os.path.join(output_dir, '{}.png'.format(rbp))
    # l2fc = [np.log2(f) for f in vals['fold_change']]
    # sunburst(vals['ip_rpr'], vals['fold_change'], vals.index, colors, output_file)
    sunburst(vals['ip_rpr'], vals['fold_change'], vals.index, colors, output_file)
    progress.update(1)

  6%|▌         | 20/362 [00:06<01:46,  3.22it/s]/home/bay001/anaconda2/envs/brian/lib/python2.7/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
 24%|██▍       | 88/362 [00:28<01:41,  2.69it/s]

Pie slices do not equal 1! Pie chart may be incomplete! outputs/278_02_PRPF8.png, 1.0


 67%|██████▋   | 242/362 [01:21<00:41,  2.91it/s]

Pie slices do not equal 1! Pie chart may be incomplete! outputs/652_02_SUGP2.png, 1.0


100%|██████████| 362/362 [02:06<00:00,  2.81it/s]

In [10]:
df.loc['203_02_HNRNPC'].sort_values(by=['ip_rpr'], ascending=False)

,RBP,cell,element,uID,uID2,ip_reads,ip_rpr,input_,input_rpr,fold_change,information_content
203_02_HNRNPC,HNRNPC,HepG2,unique_distintron,203_02_HNRNPC,HNRNPC-HepG2_02,2959472,2.298698e-01,1537674,1.371478e-01,1.676073,1.712726e-01
203_02_HNRNPC,HNRNPC,HepG2,antisense_Alu,203_02_HNRNPC,HNRNPC-HepG2_02,2464377,1.914145e-01,303999,2.711420e-02,7.059567,5.397084e-01
203_02_HNRNPC,HNRNPC,HepG2,antisense_L1,203_02_HNRNPC,HNRNPC-HepG2_02,928277,7.210167e-02,381758,3.404966e-02,2.117544,7.804227e-02
203_02_HNRNPC,HNRNPC,HepG2,antisense_RN7SL,203_02_HNRNPC,HNRNPC-HepG2_02,771448,5.992035e-02,70388,6.278029e-03,9.544454,1.950205e-01
203_02_HNRNPC,HNRNPC,HepG2,unique_proxintron,203_02_HNRNPC,HNRNPC-HepG2_02,659869,5.125372e-02,472735,4.216406e-02,1.215579,1.443526e-02
203_02_HNRNPC,HNRNPC,HepG2,Alu,203_02_HNRNPC,HNRNPC-HepG2_02,489499,3.802065e-02,173589,1.548271e-02,2.455685,4.927953e-02
203_02_HNRNPC,HNRNPC,HepG2,RN7SL,203_02_HNRNPC,HNRNPC-HepG2_02,334110,2.595118e-02,83520,7.449294e-03,3.483710,4.672834e-02
203_02_HNRNPC,HNRNPC,HepG2,unique_3utr,203_02_HNRNPC,HNRNPC-HepG2_02,307038,2.384843e-02,317789,2.834416e-02,0.841388,-5.942000e-03
203_02_HNRNPC,HNRNPC,HepG2,unique_intergenic,203_02_HNRNPC,HNRNPC-HepG2_02,274467,2.131856e-02,179612,1.601991e-02,1.330754,8.788450e-03
203_02_HNRNPC,HNRNPC,HepG2,RNA28S,203_02_HNRNPC,HNRNPC-HepG2_02,216623,1.682567e-02,2352664,2.098382e-01,0.080184,-6.125454e-02


In [11]:

len(set(df['element']))

182